In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Change fontsize from all the plots to 20
fontsize = 20
plt.rc('font', size=fontsize)

In [3]:
day_of_data = 20140101
#check if folder ./images_paper/{day_of_data} exists, if not create it
if not os.path.exists(f'./images_paper/{day_of_data}'):
    os.makedirs(f'./images_paper/{day_of_data}')

In [4]:
# Exponential model
def noise_model(x, a, alpha, b):
    return a*x**(-alpha) + b

# Frequency array
frequency = np.linspace(1/(1440*60), 1/120, 2000).astype(np.float32)

In [5]:
# Load the data - periodograms
file = f'./{day_of_data}/{day_of_data}_periodograms.h5'

with h5py.File(file, 'r') as f:
    periodograms = np.array(f['periodograms'][:])

In [ ]:
# Load the data - time series
with h5py.File(f'./{day_of_data}/{day_of_data}_data_modified.h5') as f:
    print(f.keys())
    time_series = np.array(f['time_series'][:])
    tdeltas = np.array(f['tdeltas'][:])

In [ ]:
# Load the CNN and predict the parameters for confidence
model = tf.keras.models.load_model('./CNN/Confidence/ConfModel')
model.load_weights('./CNN/Confidence/ConfModelWeights.h5')
preds = model.predict(periodograms)

scaler_params = MinMaxScaler()
scaler_params.min_, scaler_params.scale_ = np.array([1.04843577, 0.07266256, 0.6477191]),np.array([0.01904265, 0.85902252, 0.06408542])
params = scaler_params.inverse_transform(preds)
params = 10**params

In [ ]:
# Load the CNN and predict the parameters for best-fit
model_best_fit = tf.keras.models.load_model('./CNN/BestFit/BestFitModel')
model_best_fit.load_weights('./CNN/BestFit/BestFitWeights.h5')
preds_best_fit = model_best_fit.predict(periodograms)
scaler_params_best_fit = MinMaxScaler()
scaler_params_best_fit.min_, scaler_params_best_fit.scale_ = np.array([1.5661751,  0.19527236, 2.219622]), np.array([0.14172548, 2.0891168,  0.396035])
# np.array([1.69126585, 0.20132811, 2.21962195]), np.array([0.15304513, 2.15390391, 0.39603498])
params_best_fit = scaler_params_best_fit.inverse_transform(preds_best_fit)
params_best_fit = 10**params_best_fit

In [11]:
# Array of confidences
confs = np.zeros_like(periodograms)

for i in range(len(periodograms)):
    confs[i] = noise_model(frequency, *params[i])

In [12]:
# Used for going to 3d arrays from 2d stored arrays in the h5 file (periodograms)
import itertools
xrange = range(2048)
yrange = range(2048)
# Generate a list of indices
indices_list = list(itertools.product(xrange, yrange))
inside_indices_list = list(filter(lambda x: np.sqrt((x[0]-1024)**2 + (x[1]-1024)**2) <= 830, indices_list))

In [15]:
# Array of ratios psd/confidence
ratios = periodograms / confs
ratios_3d = np.zeros(shape=(2000, 2048, 2048))
for k, centr in enumerate(inside_indices_list):
    i, j = centr
    ratios_3d[:, i, j] = ratios[k]

In [16]:
#periodograms = None
confs = None
ratios = None

In [ ]:
first_frequency = 30
last_frequency = 197

ratios_3d_range = ratios_3d[first_frequency:last_frequency]
"""
This script processes a 2D array `ratios_2d` by dividing it into slices, identifying regions within each slice, and filtering these regions based on their area.
Variables:
    ratios_2d_range (ndarray): A subset of the `ratios_2d` array from index 30 to 197.
    new_ratios (ndarray): An array to store the processed slices, with shape ((167//13)+1, 2048, 2048).
Steps:
1. Iterate over 13 slices of `ratios_2d_range`.
2. For each slice:
    a. Extract a slice of 13 rows (or the remaining rows for the last slice).
    b. Compute the maximum value along the 0th axis of the slice.
    c. Create a binary mask where values greater than 1 (is psd/confidence > 1, it means there is a detection) are set to True.
    d. Label connected regions in the mask.
    e. Compute properties of the labeled regions.
    f. Calculate the 99.9th percentile of the region areas.
    g. Initialize a new slice with zeros.
    h. Iterate over the regions:
        i. Skip regions with an area smaller than the 99.9th percentile.
        ii. Copy the values of the valid regions to the new slice.
    i. Store the new slice in `new_ratios`.
    j. Print the number of valid regions in the current slice.
Dependencies:
    - numpy as np
    - skimage.measure as measure
"""
new_ratios = np.zeros(shape=((167//13)+1, 2048, 2048), dtype=np.float32)
for i in range(12+1):
    if i == 12:
        slice = ratios_3d_range[13*i:]
    else:
        slice = ratios_3d_range[13*i:13*(i+1)]

    max_slice = np.max(slice, axis=0)

    mask_slice = max_slice > 1

    print('CCL')
    label_slice = measure.label(mask_slice, connectivity=1)
    print('PROPS')
    regions_slice = measure.regionprops(label_slice)
    print('AREAS')
    areas = [region.area for region in regions_slice]
    #print(np.percentile(areas, 99))
    min_area = np.percentile(areas, 99.9)
    #mean_psd = [np.mean(max_slice[label_slice == region.label]) for region in regions_slice if region.area > 100]
    #min_area = np.percentile(areas, 95)
    #min_mean_psd = np.percentile(mean_psd, 95)
    #print(min_area, min_mean_psd)
    new_slice = np.zeros_like(max_slice)

    count = 0
    for region in regions_slice:

        #if np.max(max_slice[label_slice == region.label]) < min_mean_psd:
        #    continue
        
        if region.area < min_area:
            continue

        count += 1
        new_slice[label_slice == region.label] = max_slice[label_slice == region.label]

    print(f'Number of regions in slice {i}: {count}')
    new_ratios[i] = new_slice


In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
#import LogNorm colormap
from matplotlib.colors import LogNorm
cmap_background = plt.cm.gray
cmap_background.set_under(color='white', alpha=0)
cmap_whole = plt.cm.gist_rainbow
cmap_whole.set_under(color='white', alpha=0)  

ax.imshow(time_series[850], cmap=cmap_background, origin='lower', vmin=10)


ind = 1
vmin, vmax = 1, 6
im = ax.imshow(new_ratios[ind], cmap=cmap_whole, origin='lower', vmin=vmin, vmax=vmax)


plt.title(rf'$\nu \approx$ {round(frequency[30+(13*(ind+1))], 5):.1e} - {round(frequency[30+(13*ind)], 5):.1e} Hz / T $\approx$ {int(1/round(frequency[30+(13*(ind+1))], 5)/60)} - {int(1/round(frequency[30+(13*ind)], 5)/60)} min', fontsize=25)
cbar = fig.colorbar(im, fraction=0.046, pad=0.04)
cbar.ax.tick_params(labelsize=25)
cbar.set_label('PSD / Confidence', fontsize=25)

ax.set_xticks(range(24, 2048, 250), np.arange(24, 2048, 250)-1024)
ax.set_xlabel('X [arcsec]')
ax.set_yticks(range(24, 2048, 250), np.arange(24, 2048, 250)-1024)
ax.set_ylabel('Y [arcsec]')


ax.set_xlim(160, 2048-160)
ax.set_ylim(160, 2048-160)

x0, x1 = 1024+100, 1024+150
y0, y1 = 1024-750, 1024-800

#plt.savefig(f'./images_paper/{day_of_data}/{day_of_data}_{name}.pdf', format='pdf', dpi=300, bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.patches as patches
from skimage import exposure

cmap_background = plt.cm.gray
cmap_background.set_under(color='white', alpha=0)
cmap_whole = plt.cm.rainbow
cmap_whole.set_under(color='white', alpha=0)

# Create the figure and specify the grid dimensions
fig, axes = plt.subplots(3, 2, figsize=(15, 25))
fig.subplots_adjust(wspace=0, hspace=-0.08)

# Loop over axes and plot images
vmin, vmax = 1, 4

for i, ax in enumerate(axes.flat):
    equalized_image = exposure.equalize_adapthist(time_series[0, 150:2048, 150:2048-150], clip_limit=0.015)
    background = ax.imshow(equalized_image, cmap=cmap_background, origin='lower', vmin=0.005)
    im = ax.imshow(new_ratios[i, 150:2048, 150:2048-150], cmap=cmap_whole, origin='lower', norm=LogNorm(vmin=vmin, vmax=vmax))
    txt = ax.annotate(text='(' + chr(ord('`')+i+1) + ')',xy=(0.02, 0.92), xycoords='axes fraction')
    txt2 = ax.annotate(text=f'{int(1/frequency[30+(13*(i))]/60)} - {int(1/frequency[30+(13*(i+1))]/60)} min', xy=(0.38, 0.92), xycoords='axes fraction')


axes[0, 0].arrow(200, 1500, 0, 250, head_width=30, head_length=20)
axes[0, 0].arrow(75, 1635, 250, 0, head_width=30, head_length=20)
axes[0, 0].annotate(text='N', xy=(175, 1780), fontsize=17)
axes[0, 0].annotate(text='W', xy=(350, 1610), fontsize=17)



# CHANGE EVENTS DEPENDENT ON THE DAY OF DATA
event0 = patches.Rectangle((900-150, 875-150), 150, 150,  linewidth=1, edgecolor='k', facecolor='none')
axes[0, 1].add_patch(event0)
text0 = axes[0, 1].annotate(text='Events 1 & 2', xy=(800-150, 880), fontsize=12)

event1 = patches.Rectangle((965-150, 1075-150), 110, 125,  linewidth=1, edgecolor='r', facecolor='none')
axes[0, 1].add_patch(event1)

event2 = patches.Rectangle((565-150, 400-150), 150, 185,  linewidth=1, edgecolor='r', facecolor='none')
axes[0, 1].add_patch(event2)

event3 = patches.Rectangle((900-150, 875-150), 150, 150,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 0].add_patch(event3)

event4 = patches.Rectangle((965-150, 1075-150), 110, 125,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 0].add_patch(event4)

event5 = patches.Rectangle((565-150, 400-150), 150, 185,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 0].add_patch(event5)

event6 = patches.Rectangle((1120-150, 470-150), 170, 170,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 0].add_patch(event6)

event7 = patches.Rectangle((900-150, 875-150), 150, 150,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 1].add_patch(event7)

event8 = patches.Rectangle((1120-150, 470-150), 170, 170,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 1].add_patch(event8)

event9 = patches.Rectangle((565-150, 400-150), 150, 185,  linewidth=1, edgecolor='r', facecolor='none')
axes[1, 1].add_patch(event9)



initial_tick_pos = 324-150
final_ticks_pos = 2048-150
jump = 350
center = 1024-150
ticks_pos = range(initial_tick_pos, final_ticks_pos, jump)
ticks_labels = np.arange(initial_tick_pos, final_ticks_pos, jump) - center

axes[0, 0].set_ylabel('Y [arcsec]')
axes[0, 0].set_yticks(ticks_pos)
axes[0, 0].set_yticklabels(ticks_labels)
axes[0, 0].set_xticks(ticks_pos)
axes[0, 0].set_xticklabels([])

axes[0, 1].set_yticks(ticks_pos)
axes[0, 1].set_yticklabels([])
axes[0, 1].set_xticks(ticks_pos)
axes[0, 1].set_xticklabels([])


axes[1, 0].set_ylabel('Y [arcsec]')
axes[1, 0].set_yticks(ticks_pos)
axes[1, 0].set_yticklabels(ticks_labels)
axes[1, 0].set_xticks(ticks_pos)
axes[1, 0].set_xticklabels([])

axes[1, 1].set_yticks(ticks_pos)
axes[1, 1].set_yticklabels([])
axes[1, 1].set_xticks(ticks_pos)
axes[1, 1].set_xticklabels([])


axes[2, 0].set_ylabel('Y [arcsec]')
axes[2, 0].set_xlabel('X [arcsec]')
axes[2, 0].set_yticks(ticks_pos)
axes[2, 0].set_yticklabels(ticks_labels)
axes[2, 0].set_xticks(ticks_pos)
axes[2, 0].set_xticklabels(ticks_labels)

axes[2, 1].set_xlabel('X [arcsec]')
axes[2, 1].set_yticks(ticks_pos)
axes[2, 1].set_yticklabels([])
axes[2, 1].set_xticks(ticks_pos)
axes[2, 1].set_xticklabels(ticks_labels)


# Calculate colorbar position dynamically
left = axes[0, 0].get_position().x0
right = axes[0, -1].get_position().x1
top = axes[0, 0].get_position().y1 + 0.006  # Adjust as needed
height = 0.02  # Adjust as needed

# Add colorbar at the top of the grid
cbar_ax = fig.add_axes([left, top, right - left, height])

# Create colorbar with custom ticks and labels
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_ticks(range(vmin, vmax+1))
cbar.set_ticklabels([str(i) for i in range(vmin, vmax+1)])
cbar.ax.set_title('PSD / Confidence')
cbar_ax.xaxis.set_ticks_position('top')

plt.show()
#plt.savefig(f'./images_paper/{day_of_data}/{day_of_data}_{name}.pdf', format='pdf', dpi=400, bbox_inches='tight')

In [ ]:
from scipy.signal import windows
from astropy.timeseries import LombScargle

mini_mask = np.array(new_ratios[5,  870:980, 1180:1390] > 1)
# compute the average intensity of the pixels wheren mini-mask are true
pixel_intensity = time_series[:, 870:980, 1180:1390]

pixel_intensity = pixel_intensity[:, mini_mask].mean(axis=1)
#pixel_intensity = time_series[:, 581:591, 1300+186:1300+196].mean(axis=(1, 2))
avg = (pixel_intensity - pixel_intensity.mean())/pixel_intensity.mean()
window = windows.hann(len(tdeltas))
avg = avg*window
psd = LombScargle(tdeltas, avg, normalization='standard').power(frequency)

In [ ]:
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(13, 19))
plt.subplots_adjust(wspace=0.5, 
                    hspace=0.4)

ax0 = axes[0]

ax0.plot(tdeltas/3600, pixel_intensity, 'k')
ax0.set_ylabel('Intensity [counts]', fontsize=25)
ax0.set_xlabel('Time [hours]', fontsize=25)
ax0.set_xticks(range(0, 25, 4), np.arange(0, 25, 4), fontsize=25)
ax0.annotate('(a)', xy=(0.2, 0.9), xycoords='axes fraction', fontsize=25)

ax = axes[1]
#psd = periodograms[390792]
ax.plot(frequency, psd, color='black')
single_pred = model.predict(psd.reshape(1, -1))
single_pred = scaler_params.inverse_transform(single_pred)
single_pred = 10**single_pred

single_pred_bf = model_best_fit.predict(psd.reshape(1, -1))
single_pred_bf = scaler_params_best_fit.inverse_transform(single_pred_bf)
single_pred_bf = 10**single_pred_bf

# fill with vertical red between x1=1/122/60 and x2=1/87/60

ax.plot(frequency, noise_model(frequency, *single_pred[0]), 'r--', label='95% Confidence')
ax.plot(frequency, noise_model(frequency, *single_pred_bf[0]), 'r-', label='Best Fit')

ax.axvline(1/66/60, color='r', linestyle='--')

ax.set_xlabel('Frequency [Hz]', fontsize=25)
ax.set_ylabel('PSD', fontsize=25)


# Create a secondary x-axis on top, converting frequency to period
secax = ax.secondary_xaxis('top', functions=(lambda x: 1/x/60, lambda x: 1/x/60))
secax.set_xlabel('Period [min]', fontsize=25)

def format_func(value, tick_number):
    # Format the tick labels as desired
    if value >= 1:
        return '{:.0f}'.format(value)
    elif value >= 0.1:
        return '{:.1f}'.format(value)
    else:
        return '{:.2f}'.format(value)

secax.xaxis.set_major_formatter(FuncFormatter(format_func))


ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlim(1/(1440*60), 1/120)

for spine in ax.spines.values():
    spine.set_linewidth(1.5)  # Increase the number to make it thicker
for spine in secax.spines.values():
    spine.set_linewidth(1.5)

# Adjust the tick width and length
ax.tick_params(which='major', width=3, length=6, direction='out', labelsize=25)
secax.tick_params(which='major', width=3, length=6, direction='out', labelsize=25)

# Adjust minor ticks
ax.tick_params(which='minor', width=2, length=4, direction='out')
secax.tick_params(which='minor', width=2, length=4, direction='out')
ax.annotate('(b)', xy=(0.2, 0.9), xycoords='axes fraction', fontsize=25)
ax.legend()

plt.show()

#plt.savefig(f'./images_paper/{day_of_data}/{day_of_data}_{name}.pdf', format='pdf', dpi=400, bbox_inches='tight')